# Enviroment setup and Log in to Twitter

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
import getpass

In [ ]:
my_user = "JoanMRomero"
my_pass = getpass.getpass()

In [ ]:
search_item = "bard"

In [ ]:
PATH = " C:\Program Files\Drivers\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://twitter.com/i/flow/login")
# driver.maximize_window()
sleep(3)

In [ ]:
user_id = driver.find_element(By.XPATH, "//input[@type='text']")
user_id.send_keys(my_user)
user_id.send_keys(Keys.ENTER)

In [ ]:
password = driver.find_element(By.XPATH, "//input[@type='password']") 
password.send_keys(my_pass)
password.send_keys(Keys.ENTER)


# Scrape Tweets mentioning about "search_item"

In [ ]:
search_box = driver.find_element(By.XPATH,"//input[@data-testid='SearchBox_Search_Input']")
search_box.send_keys(search_item)
search_box.send_keys(Keys.ENTER)

In [ ]:
all_tweets = set()


tweets = driver.find_elements(By.XPATH,"//div[@data-testid='tweetText']")
while True:
    for tweet in tweets:
        all_tweets.add(tweet.text)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    sleep(3)
    tweets = driver.find_elements(By.XPATH,"//div[@data-testid='tweetText']")
    if len(all_tweets)>50:
        break

In [ ]:
all_tweets = list(all_tweets)
all_tweets[0]

# Cleaning the Tweets

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 1000
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
stp_words = stopwords.words('spanish')
print(stp_words)

In [ ]:
df = pd.DataFrame(all_tweets, columns=['tweets'])
df.head()

In [ ]:
one_tweet = df.iloc[4]['tweets']
one_tweet

In [ ]:
from textblob import TextBlob
from wordcloud import WordCloud

def TweetCleaning(tweet):
    cleanTweet = re.sub(r"@[a-zA-Z0-9]+","",tweet)
    cleanTweet = re.sub(r"#[a-zA-Z0-9\s]+","",cleanTweet)
    cleanTweet = ' '.join(word for word in cleanTweet.split() if word not in stp_words)
    return cleanTweet

def calPolarity(tweet):
    return TextBlob(tweet).sentiment.polarity

def calSubjectivity(tweet):
    return TextBlob(tweet).sentiment.subjectivity

def segmentation(tweet):
    if tweet > 0:
        return "positive"
    if tweet == 0:
        return "neutral"
    else:
        return "negative"

In [ ]:
df['cleanedTweets'] = df['tweets'].apply(TweetCleaning)
df['tPolarity'] = df['cleanedTweets'].apply(calPolarity)
df['tSubjectivity'] = df['cleanedTweets'].apply(calSubjectivity)
df['segmentation'] = df['tPolarity'].apply(segmentation)
df.head()

# Analysis and Visualization

In [ ]:
df.pivot_table(index=['segmentation'], aggfunc={'segmentation':'count'})

In [ ]:
# top 3 most positive
df.sort_values(by=['tPolarity'], ascending=False).head(3)

In [ ]:
# top 3 most negative
df.sort_values(by=['tPolarity'], ascending=True).head(3)

In [ ]:
# top 3 neutral
df[df.tPolarity==0].head(3)

In [ ]:
import matplotlib.pyplot as plt

consolidated = ' '.join(word for word in df['cleanedTweets'])

wordCloud = WordCloud(width=400, height=200, random_state=20, max_font_size=119).generate(consolidated)

plt.imshow(wordCloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
import seaborn as sns

In [ ]:
sns.set_style("whitegrid")
sns.scatterplot(data=df, x='tPolarity', y='tSubjectivity', s=100, hue='segmentation')

In [ ]:
sns.countplot(data=df, x='segmentation')